In [ ]:
# EV Routing pakcage
from ev_routing import EVRouting

# Visualization packages
import plotly.graph_objects as go

import os


# Initializing EVRouting
bl = ( 52.50, 13.34 ) # Bottom left corner coordinate
tr = ( 52.53, 13.43 ) # Top right corner coordinate

area = [ bl[0], bl[1], tr[0], tr[1] ]
evr = EVRouting( area )


# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = evr.v['lat'],
    lon = evr.v['lon'],
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 6,
        color = 'rgb(242, 177, 172)',
        opacity=0.7
    ),
    text = [ str(i) for i in range(len(evr.v)) ],
    hoverinfo = 'text'
))

nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

nodes_map.show()

In [ ]:
s = 5442
t = 9410
bs = 1e6

bt, SoC, trace = evr.dijkstra( s, t, bs )
bt

In [ ]:
trace_resolution = 50
steps = int( len(trace) / 100 ) + 1

trace_map = go.Figure()

vp = evr.v[s]

for vid in trace[ ::steps ]:
    lat = [ vp['lat'], evr.v[vid]['lat'] ]
    lon = [ vp['lon'], evr.v[vid]['lon'] ]
    vp = evr.v[vid]

    
    trace_map.add_trace( go.Scattermapbox(
        lat = lat,
        lon = lon,
        mode = 'lines',
        line = go.scattermapbox.Line(
            width = 6,
            color = 'rgb(87, 160, 211)'
        )
    ))
    
trace_map.add_trace( go.Scattermapbox(
    lat = [ evr.v[s]['lat'], evr.v[t]['lat'] ],
    lon = [ evr.v[s]['lon'], evr.v[t]['lon'] ],
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 10,
        color = 'rgb(242, 177, 172)'
    ),
    text = [ 'Start', 'End' ],
    hoverinfo = 'text'
))

trace_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

trace_map.show()